In [2]:
from lxml import etree

In [3]:
doc = etree.parse('pubmed_result.xml')

In [4]:
# import xml.etree.ElementTree as ET

# doc = ET.parse('pubmed_result.xml')

# # Using minidom

# from xml.dom import minidom

# doc = minidom.parse('pubmed_result.xml')

In [5]:
root = doc.getroot()

In [6]:
# Tag name of the root node
root.tag

'PubmedArticleSet'

In [7]:
# Fetching PMID value 
for child in root.iter('PMID'):
    print( child.text)
    pmid = child.text

27246822


In [8]:
# Fetching Article Title
for child in root.iter('ArticleTitle'):
    print( child.text)
    title = child.text

Dabrafenib plus Trametinib: a Review in Advanced Melanoma with a BRAF (V600) Mutation.


In [9]:
# Fetching Abstract Text
for child in root.iter('AbstractText'):
    print( child.text)
    annotatedAbstract = child.text

The BRAF inhibitor dabrafenib (Tafinlar(®)) and the MEK inhibitor trametinib (Mekinist(®)) are indicated, as monotherapy or in combination with each other, for the treatment of patients with unresectable or metastatic melanoma with a BRAF (V600) mutation. This article reviews the therapeutic efficacy and tolerability of combination treatment with dabrafenib and trametinib in this indication and summarizes relevant pharmacological data. Dabrafenib plus trametinib significantly prolonged progression-free survival (PFS) and overall survival (OS), improved objective response rates (ORRs) and preserved health-related quality of life (HR-QOL) to a greater extent than dabrafenib (in the double-blind COMBI-d study) and vemurafenib (in the open-label COMBI-v study) in two large, randomized, phase III studies in treatment-naïve patients with unresectable or metastatic melanoma with BRAF (V600E/K) mutation. Limited treatment benefit with the combination was also seen in patients who had progresse

In [10]:
annotatedAbstract = annotatedAbstract.encode(encoding='UTF-8',errors='strict')

## Installing grakn python client 

In [11]:
# pip install grakn-client

In [12]:
from grakn.client import GraknClient

### Writing the data inot the keyspace pubmed 

In [127]:
# Inserting 

## Connecting and creating a client
with GraknClient(uri="localhost:48555") as client:
    
    ## Creating a session
    with client.session(keyspace="pubmed") as session:
        ## creating a write transaction
        with session.transaction().write() as write_transaction:
            
            ## write transaction is open
            insert_iterator = write_transaction.query('insert $x isa pubmed-article, has pmid "{}",has title "{}" ,has annotated-abstract "{}" ;'.format(pmid,title,annotatedAbstract))           
    
            insert_list = insert_iterator.collect_concepts()
            print("Inserted pmid into the pubmed keyspace with ID: {0}".format(insert_list[0].id))
             
                     
            ## write transaction must always be committed (closed)
            write_transaction.commit()

Inserted pmid into the pubmed keyspace with ID: V40964136


# Playing with GRAKN

In [13]:
# Deleting the test doc

with GraknClient(uri='localhost:48555') as client:
    with client.session(keyspace="pubmed") as session:
        # creating a query transaction
        with session.transaction().read() as read_transaction:
            
            read_iterator = read_transaction.query('match $p isa pubmed-article, has pmid "123" ;get;')
            read_list = read_iterator.collect_concepts()
            for q in read_list:
                print(q.id, q.is_type())
        
        
        #Deleting 
        
        #del_query = write_transaction.query()

V4168 False


In [26]:
# Transaction query on doc

client = GraknClient(uri='localhost:48555')
session = client.session(keyspace="pubmed")

# creating a query transaction
query_transaction = session.transaction() 
query_iterator = query_transaction.read().query('match $p isa pubmed-article, has pmid "123" ;get;count;')#, infer= True)
#query_list = query_iterator.collect_concepts()
print(query_iterator)
for q in query_list:
    print(q.id, q.is_type())
    
query_transaction.read().close()
session.close()
client.close()
                
             

NameError: name 'query_list' is not defined

In [1]:
from grakn.client import GraknClient 

In [3]:
# Quering using Lazy execution

client = GraknClient(uri='localhost:48555')
session = client.session(keyspace="pubmed")
transaction = session.transaction().read()
transaction.query(query = 'match $p isa pubmed-article, has pmid "123" ;get;')

# Text Mining 

In [1]:
from grakn.client import GraknClient 

In [11]:
# Reading data form Grakn for text mining
client= GraknClient(uri = 'localhost:48555')
session = client.session(keyspace= 'pubmed')
transaction = session.transaction().read()
query_iterator = transaction.query('match $a isa annotated-abstract;get;')
#(query='match $p isa pubmed-article;get;')





query_concepts = query_iterator.collect_concepts()

for q in query_concepts:
    print(q,q.id,q.value)



# client = GraknClient(uri='localhost:48555')
# session = client.session(keyspace="pubmed")
# transaction = session.transaction().read()#.write()
# transaction.query(query='match $p isa pubmed-article, has pmid "123" ;get;')

<grakn.service.Session.Concept.Concept.Attribute object at 0x7f7bf4376198> V4200 <bound method Attribute.value of <grakn.service.Session.Concept.Concept.Attribute object at 0x7f7bf4376198>>
<grakn.service.Session.Concept.Concept.Attribute object at 0x7f7bf4383400> V40968232 <bound method Attribute.value of <grakn.service.Session.Concept.Concept.Attribute object at 0x7f7bf4383400>>
